<a href="https://colab.research.google.com/github/anweshaban21/DeepSectoralAI/blob/main/GRU_GARCH_BETA__SectoralAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 10.3 MB/s eta 0:00:00


In [2]:
pip install yfinance pandas numpy matplotlib


In [3]:
# Define your sectors and representative stocks
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = "2015-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


handling missing data


In [5]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()  # Forward and backward fill
    sector_data[sector] = df


Calculating Log Returns

In [6]:
import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [7]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")


In [8]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-01,486.962860,436.521210,294.023865,624.794739
2015-01-02,497.853119,442.596436,302.369690,631.223450
2015-01-05,500.999176,438.859589,302.995605,628.691589
2015-01-06,483.090820,432.027863,290.142975,621.195740
2015-01-07,482.703583,433.288757,282.297974,631.198608
...,...,...,...,...
2025-08-18,1082.599976,2003.599976,1434.599976,2001.400024
2025-08-19,1083.199951,1991.099976,1436.300049,2029.900024
2025-08-20,1080.199951,1988.199951,1430.599976,2017.500000


In [ ]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-02,0.022117,0.013821,0.027990,0.010237
2015-01-05,0.006299,-0.008479,0.002068,-0.004019
2015-01-06,-0.036400,-0.015690,-0.043344,-0.011995
2015-01-07,-0.000802,0.002914,-0.027411,0.015974
2015-01-08,0.006596,0.020788,0.026835,0.052957
...,...,...,...,...
2025-08-18,0.013391,0.006258,0.005101,0.011660
2025-08-19,0.000554,-0.006258,0.001184,0.014140
2025-08-20,-0.002773,-0.001458,-0.003976,-0.006127


In [9]:
pip install arch


In [10]:
from arch import arch_model
import pandas as pd
import numpy as np


In [11]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility


In [12]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols


Estimating GARCH volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002847. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002803. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Banking sector...
Estimating GARCH volatility for Pharma sector...
Estimating GARCH volatility for Auto sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003323. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003616. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001976. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002462. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [13]:
print(garch_volatility_data['FMCG'].tail())


            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2025-08-18  0.012895       0.011965  0.011755
2025-08-19  0.016712       0.015832  0.012331
2025-08-20  0.015747       0.015455  0.012265
2025-08-21  0.016680       0.016398  0.012296
2025-08-22  0.019554       0.015574  0.012076


LSTM

In [14]:
pip install numpy pandas scikit-learn tensorflow


In [15]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler


In [16]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import GRU, Dense



def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model





In [17]:
def predict_gru_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)  # same prepare function works
    model = build_gru_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [ ]:
gru_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting GRU volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_gru_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    gru_volatility_data[sector] = sector_vols


Predicting GRU volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


...

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
print(gru_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2025-08-18    0.001125  0.000743  0.000354  0.000444
2025-08-19    0.001140  0.000712  0.000365  0.000477
2025-08-20    0.001170  0.000689  0.000371  0.000477
2025-08-21    0.001157  0.000786  0.000433  0.000522
2025-08-22    0.001139  0.000979  0.000471  0.000603


In [ ]:
gru_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2015-02-02         NaN       NaN       NaN       NaN
2015-02-03         NaN       NaN       NaN       NaN
2015-02-04         NaN       NaN       NaN       NaN
2015-02-05         NaN       NaN       NaN       NaN
2015-02-06         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2025-07-17    0.001556  0.000694  0.000763  0.000901
2025-07-18    0.001714  0.000666  0.000812  0.000888
2025-07-21    0.001742  0.000633  0.000852  0.000833
2025-07-22    0.001780  0.000654  0.000905  0.000806
2025-07-23    0.001819  0.000688  0.000939  0.000785

[2586 rows x 4 columns]>

In [ ]:
gru_volatility_data['IT'].head()

,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2015-02-02,NaN,NaN,NaN,NaN
2015-02-03,NaN,NaN,NaN,NaN
2015-02-04,NaN,NaN,NaN,NaN
2015-02-05,NaN,NaN,NaN,NaN
2015-02-06,NaN,NaN,NaN,NaN


In [20]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']


/tmp/ipython-input-2331807048.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [21]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta



In [22]:
beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...


In [23]:
beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT         0.729562  0.817044  0.670946  0.722533     0.000000     0.000000   
Banking    0.000000  0.000000  0.000000  0.000000     1.358161     0.993063   
Pharma     0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT           0.000000      0.000000  0.000000    0.000000      0.000000   
Banking      1.337353      1.012298  0.000000    0.000000      0.000000   
Pharma       0.000000      0.000000  0.529172    0.506274      0.677415   
Auto         0.000000      0.000000  0.000000    0.000000      0.000000   
FMCG         0.000000      0.000000  0.000000    0.000000      0.000000   

           M&M.NS  MARUTI.NS  TATAMOTORS.NS  DABUR.NS  HINDUNILVR.NS    IT

Computing Sharpe Ratio

In [24]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes


In [25]:
features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            gru_mean = gru_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]
            features.append([sector, stock, beta, garch_mean, gru_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(
    features,
    columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'GRU_Vol', 'Sharpe']
)


In [26]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'GRU_Vol', 'Sharpe'])


In [27]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [ 0.0180605  -3.96206936  4.46676739]
Intercept: 0.07119651333347718


In [28]:
features_df['Risk_Score'] = X @ weights


In [29]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])



Top 3 safest stocks in IT:
        Stock  Risk_Score
0  HCLTECH.NS   -0.047549
3    WIPRO.NS   -0.045786
1     INFY.NS   -0.044933

Top 3 safest stocks in Banking:
          Stock  Risk_Score
4   AXISBANK.NS   -0.047674
6  ICICIBANK.NS   -0.041003
7  KOTAKBANK.NS   -0.040213

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
10  SUNPHARMA.NS   -0.051347
9     DRREDDY.NS   -0.049606
8       CIPLA.NS   -0.048587

Top 3 safest stocks in Auto:
            Stock  Risk_Score
13  TATAMOTORS.NS   -0.065952
11         M&M.NS   -0.047744
12      MARUTI.NS   -0.041245

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
14       DABUR.NS   -0.043776
16         ITC.NS   -0.042378
15  HINDUNILVR.NS   -0.039092


GRU AND GARCH ONLY

In [30]:
from sklearn.linear_model import LinearRegression

X_new = features_df[['GARCH_Vol', 'GRU_Vol']]
y_new = features_df['Sharpe']

model_new = LinearRegression().fit(X_new, y_new)
weights_new = model_new.coef_
intercept_new = model_new.intercept_

print("New Weights (GARCH, GRU):", weights_new)


New Weights (GARCH, GRU): [-2.41832651  2.01230715]


In [31]:
features_df['Risk_Score_GRU_GARCH'] = X_new @ weights_new


In [32]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + GRU only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_GARCH')
          .head(3)[['Stock', 'Risk_Score_GRU_GARCH']])



Safest stocks in IT (GARCH + GRU only):
        Stock  Risk_Score_GRU_GARCH
0  HCLTECH.NS             -0.037807
1     INFY.NS             -0.037181
3    WIPRO.NS             -0.036449

Safest stocks in Banking (GARCH + GRU only):
          Stock  Risk_Score_GRU_GARCH
4   AXISBANK.NS             -0.045065
6  ICICIBANK.NS             -0.040720
7  KOTAKBANK.NS             -0.036515

Safest stocks in Pharma (GARCH + GRU only):
           Stock  Risk_Score_GRU_GARCH
10  SUNPHARMA.NS             -0.039837
9     DRREDDY.NS             -0.036987
8       CIPLA.NS             -0.036367

Safest stocks in Auto (GARCH + GRU only):
            Stock  Risk_Score_GRU_GARCH
13  TATAMOTORS.NS             -0.057198
11         M&M.NS             -0.041926
12      MARUTI.NS             -0.037228

Safest stocks in FMCG (GARCH + GRU only):
            Stock  Risk_Score_GRU_GARCH
16         ITC.NS             -0.034709
14       DABUR.NS             -0.033546
15  HINDUNILVR.NS             -0.030961


GRU AND BETA ONLY

In [33]:
from sklearn.linear_model import LinearRegression

X_lb = features_df[['Beta', 'GRU_Vol']]
y_lb = features_df['Sharpe']

model_lb = LinearRegression().fit(X_lb, y_lb)
weights_lb = model_lb.coef_
intercept_lb = model_lb.intercept_

print("Weights: Beta =", weights_lb[0], ", GRU =", weights_lb[1])
print("Intercept:", intercept_lb)


Weights: Beta = 0.0009343752548602529 , GRU = -13.993203140686402
Intercept: 0.04090757491350612


In [34]:
features_df['Risk_Score_GRU_Beta'] = X_lb @ weights_lb


In [35]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (BiLSTM + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_Beta')
          .head(3)[['Stock', 'Risk_Score_GRU_Beta']])



Safest stocks in IT (BiLSTM + Beta only):
        Stock  Risk_Score_GRU_Beta
1     INFY.NS            -0.013897
0  HCLTECH.NS            -0.013851
3    WIPRO.NS            -0.009852

Safest stocks in Banking (BiLSTM + Beta only):
          Stock  Risk_Score_GRU_Beta
4   AXISBANK.NS            -0.018229
6  ICICIBANK.NS            -0.017379
7  KOTAKBANK.NS            -0.014944

Safest stocks in Pharma (BiLSTM + Beta only):
           Stock  Risk_Score_GRU_Beta
9     DRREDDY.NS            -0.021641
10  SUNPHARMA.NS            -0.019540
8       CIPLA.NS            -0.016707

Safest stocks in Auto (BiLSTM + Beta only):
            Stock  Risk_Score_GRU_Beta
13  TATAMOTORS.NS            -0.020850
11         M&M.NS            -0.016616
12      MARUTI.NS            -0.016262

Safest stocks in FMCG (BiLSTM + Beta only):
            Stock  Risk_Score_GRU_Beta
15  HINDUNILVR.NS            -0.014000
16         ITC.NS            -0.013212
14       DABUR.NS            -0.008860


GARCH AND BETA ONLY

In [36]:
from sklearn.linear_model import LinearRegression

X_gb = features_df[['Beta', 'GARCH_Vol']]
y_gb = features_df['Sharpe']

model_gb = LinearRegression().fit(X_gb, y_gb)
weights_gb = model_gb.coef_
intercept_gb = model_gb.intercept_

print("Weights: Beta =", weights_gb[0], ", GARCH =", weights_gb[1])
print("Intercept:", intercept_gb)


Weights: Beta = 0.017482033682569866 , GARCH = -3.5699063483086926
Intercept: 0.07020341037036729


In [37]:
features_df['Risk_Score_GARCH_Beta'] = X_gb @ weights_gb


In [38]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GARCH_Beta']])



Safest stocks in IT (GARCH + Beta only):
        Stock  Risk_Score_GARCH_Beta
0  HCLTECH.NS              -0.046141
1     INFY.NS              -0.043714
3    WIPRO.NS              -0.043408

Safest stocks in Banking (GARCH + Beta only):
          Stock  Risk_Score_GARCH_Beta
4   AXISBANK.NS              -0.046921
6  ICICIBANK.NS              -0.040685
7  KOTAKBANK.NS              -0.039579

Safest stocks in Pharma (GARCH + Beta only):
           Stock  Risk_Score_GARCH_Beta
10  SUNPHARMA.NS              -0.051247
9     DRREDDY.NS              -0.050444
8       CIPLA.NS              -0.048085

Safest stocks in Auto (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
13  TATAMOTORS.NS              -0.064072
11         M&M.NS              -0.046784
12      MARUTI.NS              -0.040887

Safest stocks in FMCG (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
14       DABUR.NS              -0.041450
16         ITC.NS              -0.041290
15  HINDUNILVR.NS         

In [39]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']].head()


,Stock,Risk_Score,Risk_Score_GRU_GARCH,Risk_Score_GRU_Beta,Risk_Score_GARCH_Beta
0,HCLTECH.NS,-0.047549,-0.037807,-0.013851,-0.046141
1,INFY.NS,-0.044933,-0.037181,-0.013897,-0.043714
2,TCS.NS,-0.042337,-0.033619,-0.006849,-0.039485
3,WIPRO.NS,-0.045786,-0.036449,-0.009852,-0.043408
4,AXISBANK.NS,-0.047674,-0.045065,-0.018229,-0.046921


Comparison per sector

In [40]:
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [41]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
0,HCLTECH.NS,-0.047549
3,WIPRO.NS,-0.045786
1,INFY.NS,-0.044933



🔹 Top 3 safest stocks using Risk_Score_GRU_GARCH:


,Stock,Risk_Score_GRU_GARCH
0,HCLTECH.NS,-0.037807
1,INFY.NS,-0.037181
3,WIPRO.NS,-0.036449



🔹 Top 3 safest stocks using Risk_Score_GRU_Beta:


...

,Stock,Risk_Score_GRU_GARCH
10,SUNPHARMA.NS,-0.039837
9,DRREDDY.NS,-0.036987
8,CIPLA.NS,-0.036367



🔹 Top 3 safest stocks using Risk_Score_GRU_Beta:


,Stock,Risk_Score_GRU_Beta
9,DRREDDY.NS,-0.021641
10,SUNPHARMA.NS,-0.019540
8,CIPLA.NS,-0.016707



🔹 Top 3 safest stocks using Risk_Score_GARCH_Beta:


,Stock,Risk_Score_GARCH_Beta
10,SUNPHARMA.NS,-0.051247
9,DRREDDY.NS,-0.050444
8,CIPLA.NS,-0.048085


Comparison table

In [42]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')


In [43]:
sector_ranking_table("FMCG")


,Stock,Risk_Score_Rank,Risk_Score_GRU_GARCH_Rank,Risk_Score_GRU_Beta_Rank,Risk_Score_GARCH_Beta_Rank
14,DABUR.NS,1.0,2.0,3.0,1.0
16,ITC.NS,2.0,1.0,2.0,2.0
15,HINDUNILVR.NS,3.0,3.0,1.0,3.0
